In [11]:
import pandas as pd
from models_classifier import generate_model
import matplotlib.pyplot as plt
from xgboost import plot_importance
import shap
from functions import feature_importance

In [2]:
data_path = "./data/Preprocessed_Data.xlsx"
df = pd.read_excel(data_path)
df

,Record ID,Age,Male,White,Date of Procedure,BMI,Tobacco Use,Diabetes,HbA1c Baseline,Hypertension,...,CK MA (mm)_rate_since_last_timepoint,CRT MA (mm)_difference_since_last_timepoint,CRT MA (mm)_rate_since_last_timepoint,CKH R (min)_difference_since_last_timepoint,CKH R (min)_rate_since_last_timepoint,CFF MA (mm)_difference_since_last_timepoint,CFF MA (mm)_rate_since_last_timepoint,Time to Event,Thrombosis_event,Upcoming_event
0,1,70,1,0,2017-12-04,29.728808,1,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,2,46,0,0,1999-07-12,27.547356,3,1,8,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,3,67,1,1,1997-04-25,31.391773,3,0,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,3,48,1,0,2015-10-13,31.700002,1,0,6,1,...,0.075757,-0.728237,-0.080915,0.222137,0.024682,-0.548554,-0.060950,NaN,False,False
4,4,57,1,1,2007-03-11,27.309938,2,0,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,79,50,1,1,2019-06-20,30.454834,3,0,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
146,79,59,0,1,2020-10-30,29.891027,1,1,6,0,...,0.038978,0.401491,0.133830,-0.267984,-0.089328,0.491565,0.163855,NaN,False,False
147,79,47,1,0,1997-01-08,28.158641,3,0,5,0,...,-0.216876,-0.153692,-0.051231,0.167797,0.055932,1.633006,0.544335,NaN,False,False
148,79,55,1,1,2005-04-23,27.659768,3,0,5,1,...,0.290292,-0.151138,-0.050379,0.049269,0.016423,-0.721748,-0.240583,NaN,False,False


In [3]:
# Make models to find contribution of each parameter
best_model, best_params, accuracy, X_train = generate_model(df)


is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `

In [4]:
model = best_model.named_steps['classifier']
scaler = best_model.named_steps['scaler']
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [12]:
# Get feature importances from the XGBoost model in the pipeline
feature_importance_df = feature_importance(best_model, X_train)
feature_importance_df

,Feature,Importance,Percentage Contribution
36,MAPT,0.194548,19.454813
59,CKH R (min),0.189214,18.921398
3,BMI,0.162140,16.213966
55,CK K (min),0.142886,14.288624
58,CRT MA (mm),0.088310,8.830996
...,...,...,...
34,Plavix Dose,0.000000,0.000000
33,Aspirin Dose,0.000000,0.000000
32,Bypass,0.000000,0.000000
31,DVA,0.000000,0.000000
